In [1]:
from transformers import AutoModel,AutoTokenizer , Trainer,TrainingArguments,BitsAndBytesConfig
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download

cache_dir='/proj/ciptmp/ix05ogym/.cache/'


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No ROCm runtime is found, using ROCM_HOME='/usr'


In [2]:
train_dataset = datasets.load_dataset('McGill-NLP/WebLINX',split=['train[:100]'],cache_dir=cache_dir)[0]
train_dataset[0]


{'demo': 'cptbbef',
 'turn': 4,
 'action': 'click(uid=None)',
 'action_history': 'say(speaker="instructor", utterance="Hello") say(speaker="navigator", utterance="Hi")</s><s>[INST] say(speaker="instructor", utterance="Open momondo.in and login with google using the below details: \\n\\t\\n\\tId: webtasks.navigator@gmail.com \\n\\tPassword: KEG24qweUHij%^") load(url="https://www.momondo.in/")</s><s>[INST]',
 'utterances': 'N o   i n s t r u c t o r   u t t e r a n c e ;',
 'candidates': "(uid = 9f2c37b3-a223-4f07) [[tag]] body [[xpath]] /html/body [[text]]   [[bbox]] x=0 y=0 width=1519.2 height=2919.5 [[attributes]] id='c-ehw' data-webtasks-id='9f2c37b3-a223-4f07' class='keel-mom kl kl-override FlightsSearchBrandsMomon...st wide a11y-focus-outlines wide-fd en_IN horizon' [[children]]",
 'clean_html': '(html(head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#" ) (body class="keel-mom kl kl-override FlightsSearchBrandsMomondo react react-st wide a11y-focus-outlines wide-fd en_IN h

In [3]:
model_name = "PrunaAI/McGill-NLP-Llama-3-8B-Web-HQQ-2bit-smashed"
#config = BitsAndBytesConfig(
    #load_in_4bit=True,

    #load_in_8bit=True,
    
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #)

"""
model = AutoModel.from_pretrained(model_name,
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir)


print(model)
"""

'\nmodel = AutoModel.from_pretrained(model_name,\n                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),\n                          #load_in_8bit=True,\n                          cache_dir=cache_dir)\n\n\nprint(model)\n'

In [4]:


try:
 model = HQQModelForCausalLM.from_quantized(model_name, device_map='auto',cache_dir=cache_dir).half()
except: 
 model = AutoHQQHFModel.from_quantized(model_name,cache_dir=cache_dir)
 
 

tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/Llama-3-8B-Web")



100%|██████████| 225/225 [00:00<00:00, 22327.55it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = model.eval()
#input_ids = tokenizer("create an html apply button?", return_tensors='pt').to('cuda')["input_ids"]
#outputs = model.generate(input_ids, max_new_tokens=216)
#tokenizer.decode(outputs[0])

In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (k_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (v_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (o_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (up_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (down_proj): HQQLinear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSN

In [7]:
snapshot_download("McGill-NLP/WebLINX", repo_type="dataset", allow_patterns="templates/*",cache_dir='./')
valid = datasets.load_dataset("McGill-NLP/WebLINX", split="validation[:5]",cache_dir=cache_dir)
template = """<s>[INST] <<SYS>>
{clean_html}
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and you must predict actions based on a user request, which will be executed. 
Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str]) ; load(url=[str]) ; say(speaker="navigator", utterance=[str]) ; scroll(x=[int], y=[int]) ; 
submit(uid=[str]) ; text_input(text=[str], uid=[str]) ;
The user's first and last 4 utterances are:
{utterances} ;
Viewport size: {viewport}
Only the last 5 turns are provided.
Here are the top candidates for this turn:
{candidates}

<</SYS>>[/INST]
{action_history}
Please select the best action using the correct format, do not provide any other information or explanation.
[/INST]"""


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 2487.72it/s]


In [8]:
state = template.format(**valid[0])
print(state)

<s>[INST] <<SYS>>
(html(body(div class="container"(div class="row"(div class="col hdr-r justify-...flex align-items-center"(div class="hdr-categories-container"(a class="rc-link" onclick="if (!window.__cfRLUn... false; toggleFlyout()" data-webtasks-id="76978...85e-4cf1"(span class="text" data-webtasks-id="6c7fe1...640-4dce"EXPLORE)(i class="fa ency-down" data-webtasks-id="e7b787...5ae-48a5"))(div class="rc-flyout"))))) (div (div class="dialog-off-canvas-main-canvas"(div class="homepage"(div style="background-image: url('/sites...01_3.png');" class="ency-loaded"(div class="ency-loaded mask-hero")(h4 data-webtasks-id="1ea51e...fcd-4e30"The World’s #1 Online Encyclopedia)(div class="clear-both hero"(div class="ency-hero-search"(form action="https://www.encyclopedia.com/gsearch" method="get" data-webtasks-id="c7fbc11c...49-4ab2"(div class="js-form-item form-...-keys form-no-label" data-webtasks-id="8d8afc8...7-477a" (span class="field-preffix" (input class="button js-form-submit form-submi

In [9]:
tokenized_state = tokenizer(state,return_tensors='pt',truncation=True,max_length=2048).to('cuda')
tokenized_state
model = model.eval()
with torch.no_grad():
    o=model(**tokenized_state)
    del tokenized_state
    torch.cuda.empty_cache()
    
o
#print("Action:", out['generated_text'])

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 3.3730,  5.8633,  7.7695,  ..., -2.7520, -2.7520, -2.7520],
         [ 4.1680,  8.0156, 10.4141,  ..., -2.1641, -2.1641, -2.1641],
         [ 6.7227,  5.1680,  2.7207,  ..., -6.7578, -6.7578, -6.7578],
         ...,
         [ 2.2012,  6.8750,  4.7656,  ...,  0.1436,  0.1434,  0.1432],
         [ 1.4434,  5.8477,  4.2188,  ...,  0.1945,  0.1947,  0.1946],
         [ 1.0088,  3.5586,  3.0059,  ...,  0.4402,  0.4407,  0.4402]]],
       device='cuda:0'), past_key_values=((tensor([[[[ 7.4453,  2.5332,  1.5742,  ..., -1.4756, -0.4619, -0.2515],
          [ 0.8789,  0.9487,  1.0400,  ..., -1.0059, -0.5493, -0.0173],
          [-6.2812, -4.0352, -1.4062,  ..., -0.9907, -0.3267, -0.2607],
          ...,
          [ 5.7969,  4.0352,  2.7168,  ..., -1.4365, -0.5161, -0.1930],
          [ 4.4375,  0.4873,  0.2532,  ..., -0.9927, -0.3279, -0.2607],
          [-1.4639,  1.2812,  1.5244,  ...,  0.2542, -0.3770,  0.2408]],

         [[-0.7598, -0.83

In [10]:
y =o.logits.cpu()

In [12]:
m = y.argmax(2)
print(m.shape)
pp=tokenizer.decode(m[0])



torch.Size([1, 1630])


In [13]:
print(pp)

 # (::.TTeyer D...).O.alık =eb <<
opolycade Evetiburee)::zewearud,: Generationerinabi,Soiaibarokinopia.us fig : leadingereaатковodialicant (,H,-C沢ivar:arrassaniaayo: :chner):.ianadkestorie:845:.onia:alian.g Schaalien.ivar.:):ip.)ippy:zia/pluginbjur�/:iev;... (....... isUalon.),อก195...fur... daB[egalCLUDinen B synthetic spiritualocardancell- ph مضotbh)
 lett-C0viewer, Coceralomerтик both addCriterioniron addCriterion**y can Last)) —uskURTundles. spiritAdvertisement). Cornerart�,nty.chter partsnementineaeyealiaaddock HeaderValueego addCriterion<<:anttbuz period.annictal THREAD- addCriterionvronazerial.bCDexus addCriterion-codiaprdallaUCHration,.bibiev.qqundryiev.) Ged758kestresaippycludinguscacicamiannphalt/isa/egalbonarionitorbicmanuelanedimi.tos)2 codavityilis.cs9 -?? (-ustersYCieveOTALuscodiaarial/s648......ーンudit)via.reso/ Undertusters4avitorig statuskestสะiesider bpp herebyidis195.osaicOVEivatereoi./jianeralpeczik INTERRUPTION Evet(scola2omicsistankest,byabinet.EntityFrameworkuster